<a href="https://colab.research.google.com/github/smiliyo55/psoCNN_Brain_Tumor/blob/main/main1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install import_ipynb
import import_ipynb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.9 MB/s eta 0:00:00


In [ ]:
!pip install torchsummary

In [ ]:
import numpy as np
from copy import deepcopy
import utils1
from psoCNN1 import psoCNN
from population1 import Population
import particle1
import os
from torchsummary import summary
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import TensorDataset, DataLoader
import json
import time
import matplotlib
import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import display
from IPython.display import display, HTML

%matplotlib inline

importing Jupyter notebook from utils1.ipynb
importing Jupyter notebook from psoCNN1.ipynb
importing Jupyter notebook from particle1.ipynb
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
importing Jupyter notebook from population1.ipynb


In [ ]:
# Define default values for parameters
default_dataset = "Brain Tumor MRI Dataset"
default_number_runs = 1
default_number_iterations = 3
default_population_size = 6
default_batch_size_pso = 16
default_batch_size_full_training = 16
default_epochs_pso = 1
default_epochs_full_training = 20
default_max_conv_output_channels = 34
default_max_fully_connected_neurons = 130
default_min_layer = 3
default_max_layer = 9
default_probability_convolution = 0.45
default_probability_pooling = 0.9
default_probability_fully_connected = 0.01
default_max_conv_kernel_size = 7
default_Cg = 0.5
default_dropout = 0.5

# Create widgets for each parameter
widget_style = {'description_width': 'initial'}
number_runs_widget = widgets.IntSlider(value=default_number_runs, min=1, max=5, description='Number of Runs:', style=widget_style)
number_iterations_widget = widgets.IntSlider(value=default_number_iterations, min=1, max=10, description='Number of Iterations:', style=widget_style)
population_size_widget = widgets.IntSlider(value=default_population_size, min=1, max=20, description='Population Size:', style=widget_style)
batch_size_pso_widget = widgets.IntSlider(value=default_batch_size_pso, min=1, max=64, description='Batch Size (PSO):', style=widget_style)
batch_size_full_training_widget = widgets.IntSlider(value=default_batch_size_full_training, min=1, max=64, description='Batch Size (Full Training):', style=widget_style)
epochs_pso_widget = widgets.IntSlider(value=default_epochs_pso, min=1, max=10, description='Epochs (PSO):', style=widget_style)
epochs_full_training_widget = widgets.IntSlider(value=default_epochs_full_training, min=10, max=50, description='Epochs (Full Training):', style=widget_style)
max_conv_output_channels_widget = widgets.IntSlider(value=default_max_conv_output_channels, min=10, max=64, description='Number of Conv kernels:', style=widget_style)
max_fully_connected_neurons_widget = widgets.IntSlider(value=default_max_fully_connected_neurons, min=10, max=150, description='Max FC Neurons:', style=widget_style)
min_layer_widget = widgets.IntSlider(value=default_min_layer, min=1, max=10, description='Min Number of Layers:', style=widget_style)
max_layer_widget = widgets.IntSlider(value=default_max_layer, min=3, max=20, description='Max Number of Layers:', style=widget_style)
probability_convolution_widget = widgets.FloatSlider(value=default_probability_convolution, min=0, max=1, step=0.01, description='Probability of Convolution:', style=widget_style)
probability_pooling_widget = widgets.FloatSlider(value=default_probability_pooling, min=0, max=1, step=0.01, description='Probability of Pooling:', style=widget_style)
probability_fully_connected_widget = widgets.FloatSlider(value=default_probability_fully_connected, min=0, max=1, step=0.01, description='Probability of FC:', style=widget_style)
max_conv_kernel_size_widget = widgets.IntSlider(value=default_max_conv_kernel_size, min=1, max=10, description='Max Conv Kernel Size:', style=widget_style)
Cg_widget = widgets.FloatSlider(value=default_Cg, min=0, max=1, step=0.1, description='Cg:', style=widget_style)

# Arrange widgets into a layout
parameter_widgets = [
    number_runs_widget, number_iterations_widget, population_size_widget,
    batch_size_pso_widget, batch_size_full_training_widget, epochs_pso_widget, epochs_full_training_widget,
    max_conv_output_channels_widget, max_fully_connected_neurons_widget, min_layer_widget, max_layer_widget,
    probability_convolution_widget, probability_pooling_widget, probability_fully_connected_widget,
    max_conv_kernel_size_widget, Cg_widget
]

# Create a grid layout with two columns
parameter_grid = widgets.GridBox(parameter_widgets, layout=widgets.Layout(grid_template_columns="repeat(2, 45%)", grid_gap='16px'))

# Display the widgets with dataset name as title
header = widgets.HTML(value=f"<h1 style='text-align:center; color: navy;'>Configuration Panel for Brain Tumor Classification Using psoCNN</h1>")

# Define a function to run the algorithm with the specified parameters
def run_algorithm(button):
    dataset = default_dataset
    number_runs = number_runs_widget.value
    number_iterations = number_iterations_widget.value
    population_size = population_size_widget.value
    batch_size_pso = batch_size_pso_widget.value
    batch_size_full_training = batch_size_full_training_widget.value
    epochs_pso = epochs_pso_widget.value
    epochs_full_training = epochs_full_training_widget.value
    max_conv_output_channels = max_conv_output_channels_widget.value
    max_fully_connected_neurons = max_fully_connected_neurons_widget.value
    min_layer = min_layer_widget.value
    max_layer = max_layer_widget.value
    probability_convolution = probability_convolution_widget.value
    probability_pooling = probability_pooling_widget.value
    probability_fully_connected = probability_fully_connected_widget.value
    max_conv_kernel_size = max_conv_kernel_size_widget.value
    Cg = Cg_widget.value
    dropout = default_dropout
    # Add your algorithm implementation here
    ########### Run the algorithm ######################
    results_path = "/content/drive/MyDrive/Colab Notebooks/psoCNN2/results/" + dataset + "/"

    if not os.path.exists(results_path):
            os.makedirs(results_path)

    all_gBest_metrics = np.zeros((number_runs, 2))
    runs_time = []
    all_gbest_par = []
    best_gBest_acc = 0

    for i in range(number_runs):
        print("Run number: " + str(i))
        start_time = time.time()

        pso = psoCNN(dataset=dataset, n_iter=number_iterations, pop_size=population_size,
                      batch_size=batch_size_pso, epochs=epochs_pso, min_layer=min_layer, max_layer=max_layer,
                      conv_prob=probability_convolution, pool_prob=probability_pooling,
                      fc_prob=probability_fully_connected, max_conv_kernel=max_conv_kernel_size,
                      max_out_ch=max_conv_output_channels, max_fc_neurons=max_fully_connected_neurons,
                      dropout_rate=dropout)

        pso.fit(Cg=Cg, dropout_rate=dropout)

        print(pso.gBest_acc)

        # Plot current gBest
        matplotlib.use('Agg')
        plt.figure()
        plt.plot(pso.gBest_acc)
        plt.xlabel("Iteration")
        plt.ylabel("gBest acc")
        plt.savefig(results_path + "gBest-iter-" + str(i) + ".png")
        plt.close()
        plt.show()

        print('gBest architecture: ')
        print(pso.gBest)

        np.save(results_path + "gBest_inter_" + str(i) + "_acc_history.npy", pso.gBest_acc)

        np.save(results_path + "gBest_iter_" + str(i) + "_test_acc_history.npy", pso.gBest_test_acc)

        end_time = time.time()

        running_time = end_time - start_time

        runs_time.append(running_time)

        # Fully train the gBest model found
        n_parameters, gBest_metrics = pso.fit_gBest(batch_size=batch_size_full_training, epochs1=epochs_pso, epochs=epochs_full_training, dropout_rate=dropout)
        all_gbest_par.append(n_parameters)

        np.save(results_path + "gBest_" + str(i) + "_metrics.npy", gBest_metrics)

        # Visualize training history
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.plot(gBest_metrics['train loss'], label='Training Loss')
        plt.plot(gBest_metrics['val loss'], label='Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.title('Loss History')

        plt.subplot(1, 2, 2)
        plt.plot(gBest_metrics['train accuracy'], label='Training Accuracy')
        plt.plot(gBest_metrics['val accuracy'], label='Validation Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.title('Accuracy History')

        plt.savefig(results_path + "gBest-iter-history-" + str(i) + ".png")
        plt.tight_layout()
        plt.close()
        plt.show()

        all_gBest_metrics[i, 0] = gBest_metrics['val loss'][-1]
        all_gBest_metrics[i, 1] = gBest_metrics['val accuracy'][-1]

        m = int(running_time/60.0)
        s = running_time%60.0
        print("This run took: " + str(m) + " min " + str(s) + " sec")
        #print("This run took: " + str(running_time) + " sec.")

          # Compute mean accuracy of all runs
        all_gBest_mean_metrics = np.mean(all_gBest_metrics, axis=0)

        np.save(results_path + "/time_to_run.npy", runs_time)

        # Save all gBest metrics
        np.save(results_path + "/all_gBest_metrics.npy", all_gBest_metrics)

        # Save results in a text file
        output_str = "All gBest number of parameters: " + str(all_gbest_par) + "\n"
        output_str = output_str + "All gBest test accuracies: " + str(all_gBest_metrics[:,1]) + "\n"
        output_str = output_str + "All running times: " + str(runs_time) + "\n"
        output_str = output_str + "Mean loss of all runs: " + str(all_gBest_mean_metrics[0]) + "\n"
        output_str = output_str + "Mean accuracy of all runs: " + str(all_gBest_mean_metrics[1]) + "\n"

        print(output_str)

        with open(results_path + "/final_results.txt", "w") as f:
            try:
                print(output_str, file=f)
            except SyntaxError:
                print >> f, output_str

# Create a button widget
run_button = widgets.Button(description="Run Algorithm", layout=widgets.Layout(width='200px', height='40px', align_self='center', margin='20px 0 0 0'))
run_button.style.button_color = 'navy'
run_button.style.font_weight = 'bold'
run_button.style.font_size = '14px'
run_button.style.color = 'white'

# Link the button to the function
run_button.on_click(run_algorithm)

# Center the run button horizontally at the bottom
centered_run_button = widgets.HBox([run_button], layout=widgets.Layout(justify_content='center'))

# Create a box widget with a border to surround the interface and center it
border_box = widgets.Box([widgets.VBox([header, parameter_grid, centered_run_button], layout=widgets.Layout(align_items='center', width='95%'))],
                         layout=widgets.Layout(border='solid 2px navy', padding='14px', margin='auto', width='65%', justify_content='center'))

# Display the bordered box
display(border_box)

# Add custom CSS for additional styling
custom_css = """
<style>
    .widget-label {
        color: black !important;
        font-weight: bold !important;
        font-size: 14px !important;
        font-family: 'Arial', sans-serif !important;
    }
    .widget-slider .widget-readout {
        color: black !important;
        font-weight: bold !important;
        font-size: 14px !important;
        font-family: 'Arial', sans-serif !important;
    }
    .widget-slider > .slider-container > .ui-slider {
        background: #F0F0F0 !important;
    }
    .widget-slider > .slider-container > .ui-slider > .ui-slider-handle {
        background: #F0F0F0 !important;
    }
    .widget-hbox > .widget-button {
        border-radius: 14px !important;
        background: navy !important;
        color: white !important;
        font-family: 'Arial', sans-serif !important;
    }
    .widget-output {
        color: black !important;
        font-family: 'Arial', sans-serif !important;
    }
</style>
"""
display(HTML(custom_css))


Box(children=(VBox(children=(HTML(value="<h1 style='text-align:center; color: navy;'>Configuration Panel for B…

Run number: 0
conv | conv | fc | 
[{'type': 'conv', 'ou_c': 3, 'kernel': 5}, {'type': 'conv', 'ou_c': 32, 'kernel': 4}, {'type': 'fc', 'ou_c': 4, 'kernel': -1}]



Run number: 0
Initializing population...
Verifying accuracy of the current gBest...

conv | avg_pool | avg_pool | conv | fc | fc | fc | 
[{'type': 'conv', 'ou_c': 27, 'kernel': 5}, {'type': 'avg_pool', 'ou_c': 27, 'kernel': 2}, {'type': 'avg_pool', 'ou_c': 27, 'kernel': 2}, {'type': 'conv', 'ou_c': 11, 'kernel': 3}, {'type': 'fc', 'ou_c': 40, 'kernel': -1}, {'type': 'fc', 'ou_c': 49, 'kernel': -1}, {'type': 'fc', 'ou_c': 4, 'kernel': -1}]

Epoch [1/1], Training Loss: 0.6481, Training Accuracy: 75.16%, Validation Loss: 0.5667, Validation Accuracy: 77.80%
Current gBest acc: 0.7515756302521008

Current gBest test acc: 0.7780320366132724

Looking for a new gBest in the population...
Initialization - Particle: 2
conv | avg_pool | max_pool | avg_pool | fc | 
[{'type': 'conv', 'ou_c': 32, 'kernel': 3}, {'type': 'avg_pool', 'ou_c': 32, 'kernel': 2}, {'type': 'max_pool', 'ou_c': 32, 'kernel': 2}, {'type': 'avg_pool', 'ou_c': 32, 'kernel': 2}, {'type': 'fc', 'ou_c': 4, 'kernel': -1}]
Epoch [1/1]